In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.runnables import RunnablePassthrough,RunnableParallel,RunnableSequence,RunnableLambda

In [3]:
loader = PyPDFLoader("Employee_Manual.pdf")

In [4]:
docs = loader.load()

In [4]:
print(len(docs))

92


In [95]:
print(docs[17].page_content)

11 
 
4.5 Employment Eligibility 
 
The Immigration Reform and Control Act mandates that Temple University verify the 
identity and work eligibility of all employees. Therefore, within three workdays of the start 
of employment, all employees must provide the Human Resources Department with proof of 
employment eligibility and identity, and complete and sign the I-9 form. 
 
Any individual who fails to provide necessary, original, unexpired documentation to Human 
Resources within three days of the start of employment will be suspended without pay 
pending termination. The Human Resources Department has a list of acceptable documents 
that establish identity and/or employment verification. 
 
4.6 Sponsorship of Foreign Nationals for Lawful Permanent U.S. Residence 
 
Temple University typically will sponsor foreign nationals who are full-time, regular 
members of the faculty in tenure and tenure-track academic job classifications, provided the 
Dean of the School or College and departm

In [40]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=300)
chunks = splitter.split_documents(docs)

In [12]:
print(chunks[70])

page_content='performing the essential functions of the position, with or without reasonable 
accommodation. Upon request by an applicant, Temple University will make a rea sonable 
accommodation for qualified individuals with disabilities.  
 
Jobs with exposure to lead, heavy metals, pesticides or chemicals may require baseline 
blood or urine testing. Positions with potential exposure to asbestos or other airborne 
hazards may require respiratory tests and X-rays. Other jobs may require TB and blood-' metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-07-14T10:10:58-04:00', 'author': 'Linda M. Walker', 'moddate': '2025-07-14T10:10:58-04:00', 'source': 'Employee_Manual.pdf', 'total_pages': 92, 'page': 16, 'page_label': '17'}


In [2]:
model_name ="BAAI/bge-large-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings':False}
embeddings =  HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs 
)

c:\Users\apurv\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
#chroma
vector_store = Chroma .from_documents(chunks,embeddings, collection_metadata = {"hnsw : space": "cosine"},
persist_directory = "stores3/temple_cosine3")
print("Vector storage is created.....")

Vector storage is created.....


In [42]:
#loading chroma
load_vectore_store= Chroma(persist_directory="stores3/temple_cosine3",embedding_function=embeddings)

In [43]:
retriever= load_vectore_store.as_retriever(search_type="mmr",search_kwargs={"k":6})

In [32]:
response = retriever.invoke("new hires ")

In [11]:
print(response)

[Document(id='f4cb877c-6e04-4218-99d4-34997bb9a775', metadata={'creator': 'Microsoft® Word for Microsoft 365', 'page_label': '48', 'source': 'Employee_Manual.pdf', 'moddate': '2025-07-14T10:10:58-04:00', 'creationdate': '2025-07-14T10:10:58-04:00', 'producer': 'Microsoft® Word for Microsoft 365', 'author': 'Linda M. Walker', 'total_pages': 92, 'page': 47}, page_content='regular internal recruitment responsibilities. To start this process, contact the Human \nResources Department at (215) 204-7174. To view the entire policy, go to the HR website \nhttps://www.temple.edu/hr/managers/documents/Temp_To_Hire_Policy.pdf . \n \n10.7  New Hire: Introductory/Probationary Period \n \nAll new employees must complete an introductory or probationary period. The introductory \nperiod for staff and administrative non-bargaining unit employees is six months and can be \nextended for up to another six months based on performance (as discussed with the \nemployee) during the introductory period. The pro

In [13]:
print(format_docs(response))

regular internal recruitment responsibilities. To start this process, contact the Human 
Resources Department at (215) 204-7174. To view the entire policy, go to the HR website 
https://www.temple.edu/hr/managers/documents/Temp_To_Hire_Policy.pdf . 
 
10.7  New Hire: Introductory/Probationary Period 
 
All new employees must complete an introductory or probationary period. The introductory 
period for staff and administrative non-bargaining unit employees is six months and can be 
extended for up to another six months based on performance (as discussed with the 
employee) during the introductory period. The probationary period for bargaining unit 
employees is set forth in the applicable collective bargaining agreement. This time allows 
each employee to become acquainted with his or her new position and the work 
environment. 
 
During the introductory/probationary period, the employee’s immediate supervisor will

Employees who leave the University and are rehired are treated as new e

In [12]:
def format_docs(retrieved_docs):
    context_text="\n\n".join(doc.page_content for doc in retrieved_docs )
    return context_text


In [44]:
context_chain = RunnableSequence(retriever,RunnableLambda(format_docs))

In [45]:
a = context_chain.invoke("What is the interview process ")
print(a)

40 
 
Any employee may bid for a position in a confidential manner without prior consent or 
approval from his or her supervisor. 
 
10.4  Interview Process  
 
The Human Resources Department forwards application materials that meet the stated 
requirements for a vacant position to the hiring department, which then screens the 
application material and works with Human Resources to determine which candidates are to 
be interviewed. 
 
Employees may schedule interviews during working hours as long as the interview does not 
interfere with the obligations of his/her current position. If an employee is selected for an 
interview, the employee is encouraged to discuss the posting with his or her supervisor. 
Employees who have questions about this process may contact the Human Resources 
Department. 
 
Bargaining unit employees should consult their collective bargaining agreement for further 
information.

85

83

Temple University Department of Human 
Resources Employee Manual 
Revised: J

In [ ]:
print(a)

83

identification card falsely identifying that person as being twenty-one years of age or 
older, or to obtain or attempt to obtain liquor or malt or brewed beverages by using a 
false identification card. Penalties are stated in (2) above. 
5. It is a crime intentionally, knowingly or recklessly to manufacture, make, alter, 
sell or attempt to sell an identification card falsely representing the identity, birth
